In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('OnlineRetail_cleaned.parquet')
df

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,month_year,revenue,order_count
0,573585,11001,asstd design racing car pen,2,2011-10-31 14:41:00,3.29,10000,United Kingdom,2011-10,6.58,1
1,573585,15036,assorted colours silk fan,15,2011-10-31 14:41:00,1.63,10000,United Kingdom,2011-10,24.45,1
2,573585,15039,sandalwood fan,15,2011-10-31 14:41:00,1.63,10000,United Kingdom,2011-10,24.45,1
3,573585,15056BL,edwardian parasol black,1,2011-10-31 14:41:00,12.46,10000,United Kingdom,2011-10,12.46,1
4,573585,15056N,edwardian parasol natural,1,2011-10-31 14:41:00,12.46,10000,United Kingdom,2011-10,12.46,1
...,...,...,...,...,...,...,...,...,...,...,...
526046,570715,22419,lipstick pen red,12,2011-10-12 10:23:00,0.42,18287,United Kingdom,2011-10,5.04,2
526047,570715,22866,hand warmer scotty dog design,12,2011-10-12 10:23:00,2.10,18287,United Kingdom,2011-10,25.20,2
526048,573167,23264,set of 3 wooden sleigh decorations,36,2011-10-28 09:29:00,1.25,18287,United Kingdom,2011-10,45.00,3
526049,573167,21824,painted metal star with holly bells,48,2011-10-28 09:29:00,0.39,18287,United Kingdom,2011-10,18.72,3


In [3]:
user_purchase = df.groupby(['customerid','month_year'])['revenue'].sum().reset_index()
touchpoint = pd.crosstab(user_purchase['customerid'], user_purchase['month_year']).reset_index()
months = touchpoint.columns[1:]

touchpoint_cohort = touchpoint.merge(
    df.groupby('customerid')['month_year'].min().to_frame().reset_index(),
    how = 'inner',
    on = 'customerid'
)
touchpoint_cohort_columns = list(touchpoint.columns)
touchpoint_cohort_columns.append('first_purchase_in')
touchpoint_cohort.columns = touchpoint_cohort_columns

dummy_list = []

for i in range(len(months)):
    dummy_dict = {}

    selected_month = months[i]
    prev_months = months[:i]
    next_months = months[i+1:]

    dummy_dict['total_user_count'] = touchpoint_cohort[touchpoint_cohort['first_purchase_in'] == months[i]].shape[0]

    for prev_month in prev_months:
        dummy_dict[prev_month] = np.nan

    dummy_dict[selected_month] = 1

    for next_month in next_months:
        dummy_dict[next_month] = round(
            touchpoint_cohort[
                (touchpoint_cohort['first_purchase_in'] == months[i]) & (touchpoint_cohort[next_month] >= 1)
                ].shape[0]/ dummy_dict['total_user_count'], 
            2
        )
    
    dummy_list.append(dummy_dict)

retention_cohort_columns = ['total_user_count']
retention_cohort_columns.extend(touchpoint_cohort_columns[1:14])

retention_cohort = pd.DataFrame(data = dummy_list, 
                                index = months,
                                columns = retention_cohort_columns)
retention_cohort = retention_cohort.fillna(0).reset_index().set_index(['month_year', 'total_user_count'])
retention_cohort.style.background_gradient(axis = 0, low = 0, vmax = 0.5)

,,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12
month_year,total_user_count,,,,,,,,,,,,,
2010-12,1114,1.000000,0.290000,0.260000,0.310000,0.290000,0.320000,0.290000,0.280000,0.280000,0.310000,0.300000,0.400000,0.210000
2011-01,550,0.000000,1.000000,0.170000,0.200000,0.170000,0.240000,0.220000,0.190000,0.180000,0.230000,0.250000,0.280000,0.090000
2011-02,508,0.000000,0.000000,1.000000,0.140000,0.140000,0.210000,0.200000,0.190000,0.190000,0.210000,0.190000,0.230000,0.050000
2011-03,662,0.000000,0.000000,0.000000,1.000000,0.100000,0.170000,0.140000,0.150000,0.110000,0.180000,0.160000,0.190000,0.060000
2011-04,469,0.000000,0.000000,0.000000,0.000000,1.000000,0.140000,0.130000,0.130000,0.130000,0.140000,0.140000,0.170000,0.050000
2011-05,460,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.120000,0.110000,0.110000,0.130000,0.140000,0.160000,0.060000
2011-06,425,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.100000,0.090000,0.150000,0.130000,0.190000,0.050000
2011-07,397,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.090000,0.100000,0.110000,0.130000,0.050000
2011-08,294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.120000,0.140000,0.140000,0.070000
